In [1]:
import re
import os
import pandas as pd

# Diccionario de monedas
monedas_dict = {
    "euro": "EUR", "euros": "EUR", "eur": "EUR",
    "yen": "JPY", "yenes": "JPY", "yenes japoneses": "JPY", "jpy": "JPY", "jp": "JPY",
    "dólar": "USD", "dolares": "USD", "dólares": "USD", "usd": "USD",
    "dólares estadounidenses": "USD", "dólares americanos": "USD",
    "dólares australianos": "AUD", "dolar australiano": "AUD", "dólar australiano": "AUD", "australianos": "AUD", "aud": "AUD",
    "franco": "CHF", "francos": "CHF", "francos suizos": "CHF", "chf": "CHF",
    "libra": "GBP", "libras": "GBP", "libras esterlinas": "GBP", "gbp": "GBP",
    "peso": "MXN", "pesos mexicanos": "MXN",
    "peso colombiano": "COP", "pesos colombianos": "COP",
    "yuan": "CNY", "yuanes": "CNY",
    "real brasileño": "BRL", "reales": "BRL",
    "rupia": "INR", "rupias": "INR",
    "won": "KRW",
}

def limpiar_monto(num_str: str) -> int:
    """Convierte el monto en un entero de 5 cifras."""
    limpio = re.sub(r"[^\d]", "", num_str)  # elimina todo menos dígitos
    if not limpio:
        return None
    n = int(limpio)

    # Ajustar si el número tiene más de 5 cifras (ej: 67740000 -> 67740)
    while n > 99999:
        n //= 10

    return n


def limpiar_tasa(num_str: str) -> float | None:
    """Limpia tasas eliminando comas y dejando un único punto decimal.
       Devuelve float o None si no es válida."""
    s = num_str.replace(",", "")  # quitar comas
    if s.count(".") > 1:
        partes = s.split(".")
        s = "".join(partes[:-1]) + "." + partes[-1]

    try:
        valor = float(s)
    except ValueError:
        return None

    # Filtrar valores basura (ejemplo: USD/COP entre 3000 y 5000)
    if 3000 <= valor <= 5000:
        return round(valor, 2)
    return None


def extraer_nombre(texto: str) -> str | None:
    blacklist = {
        "aqui", "aquí", "acá", "aca", "forex", "capital", "departamento",
        "divisas", "hola", "buen", "buenos", "buenas", "día", "dias", "tarde"
    }

    patron1 = re.search(
        r"(?:^|\b)(?:soy|le habla|habla|al habla|le saluda|les saluda)\s+([A-ZÁÉÍÓÚÑ][a-záéíóúñ]+)",
        texto,
        re.IGNORECASE
    )
    if patron1:
        candidato = patron1.group(1).strip().capitalize()
        if candidato.lower() not in blacklist:
            return candidato

    patron_aqui = re.search(
        r"\b(?:aquí|aqui|acá|aca)\s+([A-ZÁÉÍÓÚÑ][a-záéíóúñ]+)\b",
        texto,
        re.IGNORECASE
    )
    if patron_aqui:
        candidato = patron_aqui.group(1).strip().capitalize()
        if candidato.lower() not in blacklist:
            return candidato

    patron_saludo = re.search(
        r"\b(?:muy\s+)?buen(?:os|as)?\s+(?:d[ií]a|tarde|noches?)\s*,?\s*([A-ZÁÉÍÓÚÑ][a-záéíóúñ]+)",
        texto,
        re.IGNORECASE
    )
    if patron_saludo:
        candidato = patron_saludo.group(1).strip().capitalize()
        if candidato.lower() not in blacklist:
            return candidato

    patron2 = re.search(
        r"\b([A-ZÁÉÍÓÚÑ][a-záéíóúñ]+)(?=\s+(?:de\s+Forex\s+Capital|al\s+habla))",
        texto,
        re.IGNORECASE
    )
    if patron2:
        candidato = patron2.group(1).strip().capitalize()
        if candidato.lower() not in blacklist:
            return candidato

    return None


# Carpeta con transcripciones
carpeta = r"C:\Users\onnyx\Desktop\Audios\Texto\TranscripcionesH"

registros = []

for archivo in os.listdir(carpeta):
    if archivo.lower().endswith(".txt"):
        ruta = os.path.join(carpeta, archivo)

        try:
            id_llamada = int(os.path.splitext(os.path.basename(ruta))[0].split("_")[1])
        except Exception:
            id_llamada = os.path.splitext(os.path.basename(ruta))[0]

        with open(ruta, "r", encoding="utf-8") as f:
            texto = f.read()

        nombre = extraer_nombre(texto)

        # Paso 1: Detectar tipo de operación con regex robusto
        tipo_operacion = None
        if re.search(r"\bcompr\w*\b", texto, re.IGNORECASE):
            tipo_operacion = "comprar"
        elif re.search(r"\bvend\w*\b", texto, re.IGNORECASE):
            tipo_operacion = "vender"

        # Paso 2: Sinónimos SOLO si tipo_operacion quedó vacío
        if tipo_operacion is None:
            if re.search(r"\b(ofertar|liquidar|desprender|ceder|colocar|tener)\b", texto, re.IGNORECASE):
                tipo_operacion = "vender"
            elif re.search(r"\b(adquirir|tomar|obtener|entrar|captar)\b", texto, re.IGNORECASE):
                tipo_operacion = "comprar"

        # Extraer números crudos
        nums_raw = re.findall(r"\d+(?:[.,]\d+)*", texto)

        # Procesar monto (primer número)
        monto_numerico = limpiar_monto(nums_raw[0]) if len(nums_raw) > 0 else None

        # Procesar tasas y filtrar solo válidas
        tasas = [limpiar_tasa(n) for n in nums_raw]
        tasas = [t for t in tasas if t is not None]

        tasa_inicial = tasas[0] if len(tasas) > 0 else None
        tasa_final = tasas[-1] if len(tasas) > 1 else None

        # Calcular spread (tasa_inicial - tasa_final)
        spread = None
        if tasa_inicial is not None and tasa_final is not None:
            spread = round(tasa_inicial - tasa_final, 2)

        # Detectar moneda
        moneda = None
        for desc, code in monedas_dict.items():
            if re.search(desc, texto, re.IGNORECASE):
                moneda = code
                break

        registros.append({
            "id": id_llamada,
            "nombre": nombre,
            "tipo_operacion": tipo_operacion,
            "monto_numerico": monto_numerico,
            "tasa_inicial": tasa_inicial,
            "tasa_final": tasa_final,
            "moneda": moneda,
            "spread": spread,
            "texto_llamada": texto.strip()
        })

df = pd.DataFrame(registros)
df

,id,nombre,tipo_operacion,monto_numerico,tasa_inicial,tasa_final,moneda,spread,texto_llamada
0,1,Sofía,comprar,16073,3980.64,3956.12,USD,24.52,"Buenos días, soy Sofía de Forex Capital, podem..."
1,10,Laura,vender,52898,4431.31,4414.91,JPY,16.40,"Muy buenos días, le habla Laura de Forex Capit..."
2,100,Sara,comprar,80951,4261.25,4239.38,USD,21.87,"Hola, soy Sara Gómez, representante de Forex C..."
3,101,Sofía,comprar,89228,4091.37,4064.83,GBP,26.54,"Hola, habla Sofía de Forex Capital. Le interes..."
4,102,Laura,vender,71042,3914.56,3879.36,USD,35.20,"Hola, soy Laura de Forex Capital. ¿Está dispon..."
...,...,...,...,...,...,...,...,...,...
495,95,Sara,vender,63264,4297.95,4253.03,GBP,44.92,"Buen día, aquí Sara Márquez de Forex Capital, ..."
496,96,Sara,comprar,82704,4374.15,4356.44,GBP,17.71,"Hola, le saluda Sara Márquez del Departamento ..."
497,97,Sara,comprar,77603,3996.87,3971.85,JPY,25.02,"Hola, habla Sara Gómez de Forex Capital. Le in..."
498,98,Juliana,comprar,35312,4122.79,4088.46,AUD,34.33,Les saluda Juliana de Forex Capital. Estamos o...


In [2]:
# Archivo base
nombre_archivo = "resultado_transcripciones"

# Exportar a Excel
salida_excel = os.path.join(carpeta, f"{nombre_archivo}.xlsx")
df.to_excel(salida_excel, index=False, engine="openpyxl")
print(f"Archivo Excel exportado en: {salida_excel}")

# Exportar a CSV con separador ":" y sin índice en la misma carpeta
salida_csv = os.path.join(carpeta, f"{nombre_archivo}.csv")
df.to_csv(salida_csv, sep=":", index=False, encoding="utf-8")
print(f"Archivo CSV exportado en: {salida_csv}")

Archivo Excel exportado en: C:\Users\onnyx\Desktop\Audios\Texto\TranscripcionesH\resultado_transcripciones.xlsx
Archivo CSV exportado en: C:\Users\onnyx\Desktop\Audios\Texto\TranscripcionesH\resultado_transcripciones.csv
